In [10]:
from bs4 import BeautifulSoup as bs
import json

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

In [11]:
f = open('beers_to_scrape.json')
dict_beers = json.load(f)

In [ ]:
for beer_type in dict_beers:
    for beer_url in dict_beers[beer_type]:

        DRIVER_PATH = "chromedriver_windows.exe"
        driver = webdriver.Chrome(executable_path=DRIVER_PATH)
        driver.get(beer_url)

        # Only need to perform this steps on the first page
        sleep(5)
        # Need to click the reject all cookies in order to show the other buttons beneath it 
        try:
            driver.find_element_by_id("onetrust-reject-all-handler").click()
        except Exception:
            pass

        # Click the button to expand out the number of reviews which we then click to 100
        divs = driver.find_elements_by_tag_name("div")
        for div in divs:
            try:
                if div.get_attribute("aria-haspopup") == "listbox":
                    div.click()
                    break
            except Exception:
                pass

        # set the number of reviews per page to 100
        rows_button = driver.find_elements_by_tag_name("li")
        for rows in rows_button:
            try:
                if rows.get_attribute("data-value") == "100":
                    rows.click()
            except Exception:
                pass
        # wait to load page again
        sleep(5)

        # This is so we can scroll back to the top of the page
        htmlelement= driver.find_element_by_tag_name('html')
        meta_data = None

        reviews = []
        for i in range(0,5):
            #scroll back to the top of the page so we can pull the meta data
            htmlelement.send_keys(Keys.HOME)

            print(f"Loading page {i+1}")

            # exapand out all the "Show more" buttons
            buttons = driver.find_elements_by_tag_name("button")

            #scroll back to the top of the page
            for button in buttons:
                if "Show more" in button.text:
                    try:
                        button.click()
                    except Exception as e:
                        print(e)
            

            # Pull out all of the review blocks + meta data
            soup = bs(driver.page_source, "html.parser")

            if not meta_data:
                meta_data = {"meta_data_html":
                    str(soup.find("div", {"class": "MuiPaper-root Paper___StyledMPaper-MfXTu eRPbAA BeerCard___StyledPaper-qAdWi jihfcX MuiPaper-elevation1 MuiPaper-rounded"}))}


            review_blocks = soup.find_all("div", {"class": "MuiTypography-root Text___StyledTypographyTypeless-bukSfn pzIrn colorized__WrappedComponent-hrwcZr hwjOn BeerReviewListItem___StyledText-kMbsdb gCtEHi pre-wrap MuiTypography-body1"})
            for i in review_blocks:
                reviews.append(i.text)

            # try to go to the next page
            next_page = False
            try:
                for button in buttons:
                    if "Next page" in str(button.get_attribute("title")):
                        next_page = True
                        button.click()
            except Exception:
                break

            if not next_page:
                break
        
        #create directories and write out the files
        beer_str = beer_url.replace("https://www.ratebeer.com/beer/", "")
        beer_str = beer_str.split("/")[0]
        path = f"beer_data/{beer_type}/{beer_str}"

        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        
        with open(f"{path}/reviews.json", "w") as fp:
            json.dump({"reviews": reviews}, fp)

        with open(f"{path}/meta_data_html.json", "w") as fp:
            json.dump(meta_data, fp)

        print(f"Done scraping {beer_str}")
        driver.close()